In [ ]:
from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from pyspark.ml import Pipeline
import pandas as pd
import re
pd.set_option("display.max_colwidth", 0)

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
builder = SparkSession.builder\
        .appName("Spark NLP Licensed")\
        .master("local[*]")\
        .config("spark.driver.memory", "24G")\
        .config("spark.driver.maxResultSize", "2048GB")\
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
        .config("spark.kryoserializer.buffer.max", "2000M")\
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.2")

spark = builder.getOrCreate()

In [ ]:
spark

In [ ]:
# textFiles = spark.sparkContext.wholeTextFiles("oncology_notes/*")
# 
# df = textFiles.toDF(schema=['path','text'])
# df.show()

In [ ]:
df = spark.read.text('text_all_word.txt').toDF('text')

In [ ]:
df.select("text").head(1)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df = df.repartition(200)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
sentencerDL = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl", "xx") \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

model = PipelineModel(stages=[documenter, sentencerDL])

In [ ]:
result = model.transform(df)

In [ ]:
pd_res = result.select("path","sentences.result").toPandas()
pd_res

In [ ]:
pd_res.loc[0].result

In [ ]:
def clean_text(text):
    text = " ".join(text)
    text_clean_brackets = text.strip("-").strip("*").strip("/").strip("+").strip("#").strip("$")
    text_clean_data_brackets = re.sub('[\(\[\{].*?[\)\]\}]', '', text_clean_brackets)
    text_clean_list = re.findall(r'\w+.', text_clean_data_brackets)
    text_result = (" ".join(text_clean_list)).lower()
    return text_result 